In [ ]:
# ! git clone https://github.com/libindic/indic-trans.git
# ! cd indictrans && pip install -r requirements.txt && pip install .

In [1]:
from Lemmatization.lemmatizer import document_linearization
from json import load
import numpy as np
import time
import sys
from helper import read_train_data, load_train_data, load_concept_data, read_concept_data
import lsa
import tfidf
import esa
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khanm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
e:\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-18:11:35:35,294 INFO     [iwn.py:43] Loading bengali language synsets...
2023-12-18:11:39:40,633 INFO     [iwn.py:43] Loading bengali language synsets...
2023-12-18:11:39:48,310 INFO     [iwn.py:43] Loading bengali language synsets...


In [ ]:
trainDatasetPath = "../../Datasets/Anwesha/Train_Dataset/"
testDatasetPath = "../../Datasets/Anwesha/Test_Collection/"
conceptDatasetPath = "../../Datasets/Anwesha/New_Dataset_Concepts/"
docs, file_paths, dids = read_train_data(testDatasetPath, save_path=True)
c_docs, c_file_paths, c_dids = read_concept_data(conceptDatasetPath, save_path=True)

In [ ]:
TFIDF_vectorizer, tfidf_matrix = tfidf.set_up_TFIDF(docs, lemmatiser=False, save_model=True)
c_vectorizer, c_tfidf_matrix, doc_concept_matrix = esa.set_up_ESA(c_docs, TFIDF_vectorizer, tfidf_matrix, lemmatiser=False, save_model=True)
LSA_vectorizer, LSA_svd_transformer, svd_model, LSA_dvecs = lsa.set_up_LSA(docs, rank=600, lemmatiser=False, save_model=True)

TF-IDF model is saved in SavedModels directory.
ESA model is saved in SavedModels directory.


In [15]:
import glob
file_names = glob.glob('D:\\Data\\repos\\Bangla-IR-eval\\Datasets\\Anwesha\\Test_Collection\\**\\*.txt', recursive=True)
file_names.extend(glob.glob('D:\\Data\\repos\\Bangla-IR-eval\\Datasets\\Anwesha\\Train_Dataset\\**\\*.txt', recursive=True))
files = {}
lit_docs = []
health_docs = []
travel_docs = []
news_docs = []
for file_name in file_names:
    doc_id = file_name.split('\\')[-1].split('.')[0]
    if "Rabindra_Rachnabali" in file_name:
        lit_docs.append(doc_id)
    elif "health" in file_name:
        health_docs.append(doc_id)
    elif "travel" in file_name:
        travel_docs.append(doc_id)
    else:
        news_docs.append(doc_id)
    
    with open(file_name, 'r', encoding='utf-8') as f:
        text = f.read()
        files[doc_id] = text
        
print("Total docs: ", len(files))
print("Literature docs: ", len(lit_docs))
print("Health docs: ", len(health_docs))
print("Travel docs: ", len(travel_docs))
print("News docs: ", len(news_docs))

Total docs:  2182
Literature docs:  182
Health docs:  144
Travel docs:  214
News docs:  1642


In [3]:
import json
with open('../../Datasets/Anwesha/qrels-merged.json', 'r') as f:
    qrels = json.load(f)

queries = qrels.keys()
print(len(queries))       

202


In [11]:
qrels_copy = qrels.copy()
for q in queries:
    doc_ids = qrels_copy[q].keys()
    for doc_id in doc_ids:
        if doc_id not in files.keys():
            print(q, doc_id)
            qrels[q].pop(doc_id)

In [17]:
lit_qrels = {}
health_qrels = {}
travel_qrels = {}
news_qrels = {}
for q in qrels.keys():
    for doc_id in qrels[q].keys():
        if doc_id in lit_docs:
            if q not in lit_qrels.keys():
                lit_qrels[q] = {}
            lit_qrels[q][doc_id] = qrels[q][doc_id]
        elif doc_id in health_docs:
            if q not in health_qrels.keys():
                health_qrels[q] = {}
            health_qrels[q][doc_id] = qrels[q][doc_id]
        elif doc_id in travel_docs:
            if q not in travel_qrels.keys():
                travel_qrels[q] = {}
            travel_qrels[q][doc_id] = qrels[q][doc_id]
        else:
            if q not in news_qrels.keys():
                news_qrels[q] = {}
            news_qrels[q][doc_id] = qrels[q][doc_id]

print("=================================================")
print("Total queries: ", len(qrels))
print("Literature queries: ", len(lit_qrels))
print("Health queries: ", len(health_qrels))
print("Travel queries: ", len(travel_qrels))
print("News queries: ", len(news_qrels))
print("=================================================")
print("Toral qrels: ", sum([len(qrels[q]) for q in qrels.keys()]))
print("Literature qrels: ", sum([len(lit_qrels[q]) for q in lit_qrels.keys()]))
print("Health qrels: ", sum([len(health_qrels[q]) for q in health_qrels.keys()]))
print("Travel qrels: ", sum([len(travel_qrels[q]) for q in travel_qrels.keys()]))
print("News qrels: ", sum([len(news_qrels[q]) for q in news_qrels.keys()]))

Total queries:  202
Literature queries:  67
Health queries:  36
Travel queries:  40
News queries:  134
Toral qrels:  3059
Literature qrels:  601
Health qrels:  194
Travel qrels:  330
News qrels:  1934


In [26]:
import os
## literature
with open('literature_qrels.json', 'w') as f:
    json.dump(lit_qrels, f)
os.mkdir('literature')
for d in lit_docs:
    with open('literature/'+d+'.txt', 'w', encoding='utf-8') as f:
        f.write(files[d])

## health
with open('health_qrels.json', 'w') as f:
    json.dump(health_qrels, f)
os.mkdir('health')
for d in health_docs:
    with open('health/'+d+'.txt', 'w', encoding='utf-8') as f:
        f.write(files[d])

## travel
with open('travel_qrels.json', 'w') as f:
    json.dump(travel_qrels, f)
os.mkdir('travel')
for d in travel_docs:
    with open('travel/'+d+'.txt', 'w', encoding='utf-8') as f:
        f.write(files[d])

## news
with open('news_qrels.json', 'w') as f:
    json.dump(news_qrels, f)
os.mkdir('news')
for d in news_docs:
    with open('news/'+d+'.txt', 'w', encoding='utf-8') as f:
        f.write(files[d])

In [11]:
from ranx import Qrels, Run, evaluate

run_dict = {}
for row in df_queries.itertuples():
    q_id = row.query_id
    q = row.query
    A, _ = tfidf.tfidf(TFIDF_vectorizer, tfidf_matrix, q, docs)
    k = 10
    x = np.argsort(A[0]).T[::-1][:k]
    index = 0
    q_dict = {}
    for y in x:
        if index < k:
            if A[0][y] > 0:
                q_dict['D'+str(dids[y][1])] = A[0][y]
        index = index + 1
    run_dict[q_id] = q_dict
run = Run(run_dict, name="anwesha-test")
qrel_df = pd.read_csv('../../Datasets/Anwesha/qrels.csv')
qrel_df['relevance'] = qrel_df['relevance'].astype('int64')
qrel = Qrels.from_df(
    df=qrel_df,
    q_id_col="query_id",
    doc_id_col="doc_id",
    score_col="relevance",
)

কালো কুমড়ো টাটকা দেখিয়ে দেব লবডঙ্কা
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.10164614 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.11684365 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.16092093 0.         0.         0.14317849 0.
  0.         0.20276824 0.         0.         0.         0.19512117
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.19307796 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.33333333 0.        